In [56]:
import numpy as np
from sklearn.cross_validation import cross_val_score, train_test_split, KFold
from sklearn.utils import shuffle


## 1.1 Dataset Preparration

In [57]:
from sklearn.datasets import load_digits
digits = load_digits()
data = digits["data"]
images = digits["images"]
target = digits["target"]
target_names = digits["target_names"]

In [75]:
Xk = []
std = data.std()
for i in range(10):
    Xk.append(data[target == i])
    Xk_mean = sum(Xk[i]) / len(Xk[i])
    Xk[i] -= Xk_mean
    data[target == i] -= Xk_mean
    Xk[i] /= std
    
data /= std

data, target = shuffle(data, target, random_state=0)

## 1.2 One vs Rest

In [77]:
from sklearn.ensemble import RandomForestClassifier as rfc

rfc_auto = []
rfc_subs = []
for i in range(10):
    rfc_auto.append(rfc(n_estimators=20, random_state=1, class_weight="auto"))
    # just returns 1.s and  0.s??? What about those weights??
    rfc_subs.append(rfc(n_estimators=20, random_state=1, class_weight="balanced_subsample"))


def binary_prediction(rfcs, X):
    bin_codes = np.matrix([rfcs[i].predict(X) for i in range(10)])
    # print("shape bincodes:", np.shape(bin_codes))
    predictions = np.where(bin_codes)
    out = np.ndarray(len(X))
    # out.fill(-1)
    out[predictions[1]] = predictions[0]
    for i in range(len(X)):
        if sum(bin_codes[:, i]) != 1:
            out[i] = -1
            # print("sum", np.sum(bin_codes[:, i]))
            # print("shape(bin_codes[:, i]", np.shape(bin_codes[:, i]))
            # print("shape(bin_codes)", np.shape(bin_codes))
            # print(bin_codes[:, i])
            # print("")
    return out
    
def argmax_prediction(rfcs, X):
    results = np.matrix([rfcs[i].predict(X) for i in range(10)])
    out = np.argmax(results, axis=0)
    return out

y_pred_auto_binary = np.empty((0), dtype=np.float_)
y_pred_subs_binary = np.empty((0), dtype=np.float_)
y_pred_auto_argmax = np.empty((0), dtype=np.float_)
y_pred_subs_argmax = np.empty((0), dtype=np.float_)
y_test = np.empty((0), dtype=np.float_)

k_fold = KFold(n=len(target), n_folds=5, shuffle=False) # shuffled the training set before with seeding
for train_indices, test_indices in k_fold:
    # print(np.shape(data[train_indices]))
    # print(target[train_indices] == i)
    for i in range(10):
        rfc_auto[i].fit(data[train_indices], target[train_indices] == i)
        rfc_subs[i].fit(data[train_indices], target[train_indices] == i)
    
    
    y_pred_auto_binary = np.append(y_pred_auto_binary, binary_prediction(rfc_auto, data[test_indices]))
    y_pred_subs_binary = np.append(y_pred_subs_binary, binary_prediction(rfc_subs, data[test_indices]))
    y_pred_auto_argmax = np.append(y_pred_auto_argmax, argmax_prediction(rfc_auto, data[test_indices]))
    y_pred_subs_argmax = np.append(y_pred_subs_argmax, argmax_prediction(rfc_subs, data[test_indices]))
    
    y_test = np.append(y_test, target[test_indices])
    

# only small differences between the methods!?
print(np.sum(y_pred_auto_binary != y_test))
print(np.sum(y_pred_subs_binary != y_test))
print(np.sum(y_pred_auto_argmax != y_test))
print(np.sum(y_pred_subs_argmax != y_test))

print(y_pred_auto_binary)
print(y_pred_subs_binary)
print(y_pred_auto_argmax)
print(y_pred_subs_argmax)
print(y_test)

1
2
1
2
[ 1.  6.  2. ...,  6.  6.  7.]
[ 1.  6.  2. ...,  6.  6.  7.]
[ 1.  6.  2. ...,  6.  6.  7.]
[ 1.  6.  2. ...,  6.  6.  7.]
[ 1.  6.  2. ...,  6.  6.  7.]
